In [3]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# ---------------------------
# Configuration / Parameters
# ---------------------------
ARMS = ["Top Banner", "Sidebar", "Footer", "Pop-up"]
# True CTRs (unknown to the algorithm). Change these to simulate different scenarios.
TRUE_CTRS = [0.05, 0.08, 0.02, 0.10]

EPSILON = 0.1           # exploration probability (10%)
N_IMPRESSIONS = 1000    # number of impressions per run
SEED = None             # set to an int for reproducible runs, or None

OUTPUT_DIR = "mab_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [4]:
# ---------------------------
# Epsilon-Greedy implementation
# ---------------------------
def run_epsilon_greedy(true_ctrs, epsilon=0.1, n_impressions=1000, seed=None):
    """
    Run one epsilon-greedy simulation.

    Returns:
        summary_df: pandas.DataFrame with per-arm stats (times selected, estimated ctr, clicks)
        selection_history: list of selected arm indices (len = n_impressions)
        reward_history: list of rewards (0/1)
        cumulative_rewards: list of cumulative rewards over time
    """
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    n_arms = len(true_ctrs)
    counts = [0] * n_arms            # times each arm selected
    values = [0.0] * n_arms          # estimated CTRs (running averages)
    selection_history = []
    reward_history = []
    cumulative_rewards = []

    total_reward = 0

    for t in range(1, n_impressions + 1):
        # Explore with prob epsilon, otherwise exploit
        if random.random() < epsilon:
            chosen = random.randrange(n_arms)
        else:
            max_val = max(values)
            # tie-breaking randomly among max-valued arms
            candidates = [i for i, v in enumerate(values) if v == max_val]
            chosen = random.choice(candidates)

        # simulate reward (click) according to true CTR
        reward = 1 if random.random() < true_ctrs[chosen] else 0

        # online update for estimated CTR (incremental average)
        counts[chosen] += 1
        values[chosen] += (reward - values[chosen]) / counts[chosen]

        selection_history.append(chosen)
        reward_history.append(reward)
        total_reward += reward
        cumulative_rewards.append(total_reward)

    # Build summary DataFrame
    total_clicks_per_arm = [0] * n_arms
    for idx, r in zip(selection_history, reward_history):
        total_clicks_per_arm[idx] += r

    summary_df = pd.DataFrame({
        "Arm": ARMS,
        "True_CTR": true_ctrs,
        "Times_Selected": counts,
        "Estimated_CTR": [round(v, 6) for v in values],
        "Clicks": total_clicks_per_arm,
        "Observed_CTR": [round(total_clicks_per_arm[i] / counts[i], 6) if counts[i] > 0 else 0.0 for i in range(n_arms)]
    })

    return summary_df, selection_history, reward_history, cumulative_rewards